# **Plant Disease**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.models import Model,Sequential
from keras.layers import Input,Conv2D,MaxPooling2D,Dropout,Flatten,Dense,GlobalAveragePooling2D,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint

from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input

from sklearn.metrics import confusion_matrix,classification_report

### **Data Generation**

In [ ]:
gentrain = Imgen(preprocessing_function=preprocess_input,
                
                shear_range = 0.2,
                zoom_range = 0.2,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                fill_mode="nearest",
                validation_split=0.15)


gentest = Imgen(preprocessing_function=preprocess_input)

In [ ]:
traindata = gentrain.flow_from_directory("input/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                      target_size=(150,150),
                                       class_mode="categorical",
                                       seed=123,
                                       batch_size=32,
                                       subset="training"
                                      )
valdata = gentrain.flow_from_directory("input/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                      target_size=(150,150),
                                       class_mode="categorical",
                                       seed=123,
                                       batch_size=32,
                                   subset="validation"
                                      )
testdata = gentest.flow_from_directory("input/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                    target_size=(150,150),
                                    class_mode="categorical",
                                    seed=123,
                                    batch_size=32,
                                    shuffle=False)

**Classes**

In [ ]:
c = traindata.class_indices
classes = list(c.keys())
classes

In [ ]:
plt.figure(figsize=[20,8])
sns.countplot(x=traindata.classes);

**One Batch**

In [ ]:
x,y = next(traindata)

def plotImages(x,y):
    plt.figure(figsize=[15,11])
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.imshow(x[i])
        plt.title(classes[np.argmax(y[i])])
        plt.axis("off")
    plt.show()

In [ ]:
plotImages(x,y)

## **Model**

In [ ]:
base_model = keras.applications.DenseNet201(include_top=False,
                     input_shape=(150,150,3),
                      weights = "imagenet",
                      pooling="avg"
                     )
base_model.trainable = False

In [ ]:
image_input = Input(shape=(150,150,3))

x = base_model(image_input,training = False)

x = Dense(512,activation = "relu")(x)

x = Dropout(0.3)(x)

x = Dense(128,activation = "relu")(x)

image_output = Dense(38,activation="softmax")(x)

model = Model(image_input,image_output)

In [ ]:
model.summary()

**Compile**

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

**Callbacks**

In [ ]:
my_calls = [EarlyStopping(monitor="val_accuracy",patience=3),
            ModelCheckpoint("Model.h5",verbose= 1 ,save_best_only=True)]

**Train**

In [ ]:
hist = model.fit(trainds,epochs=1,validation_data=valdata,callbacks=my_calls)

**Test**

In [ ]:
model.evaluate(testdata)

In [ ]:
plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(hist.epoch,hist.history['accuracy'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.epoch,hist.history['loss'],label = 'Training')
plt.plot(hist.epoch,hist.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()

## **Predictions**

In [ ]:
pred = model.predict(testdata)

In [ ]:
pred = [np.argmax(i) for i in pred]

In [ ]:
y_test = testdata.classes

**Classification Report**

In [ ]:
print(classification_report(pred,y_test))

**Confusion Matrix**

In [ ]:
plt.figure(figsize=[20,15])
sns.heatmap(confusion_matrix(pred,y_test),annot = True, fmt = "d", cmap = "viridis");